In [3]:
import json
import pandas as pd

In [24]:
# Read data from tsv into dataframe
trainData = pd.read_csv ('../data/LIAR-PLUS/dataset/tsv/train2.tsv', sep='\t', header=None)


# Add collumn names in the data.
columnNames = ['id', 'json_file_id', 'label', 'claim', 'topics', 'originator', 'title', 'state', 'party','c1', 'c2', 'c3', 'c4', 'c5', 'location', 'justification']
trainData.columns = columnNames

In [25]:
# Transform truth values from six classes to two. 'True' and 'False'
def transformTruthValues(dataFrame, trueLabels=[], falseLabels=[]):
    
    trueList  = []
    falseList = []

    for index, row in dataFrame.iterrows():
        if row['label'] in trueLabels:
            dataFrame.at[index, 'label'] = 'true'
            trueList.append(dataFrame.iloc[index].values)
        else:
            dataFrame.at[index, 'label'] = 'false'
            falseList.append(dataFrame.iloc[index].values)

    trueData  = pd.DataFrame(trueList,  columns=columnNames)
    falseData = pd.DataFrame(falseList, columns=columnNames)
    
    return trueData, falseData

In [26]:
# Fix dataframe ids to start from 0
def fixIds(dataFrame):    
    id = 0
    for index, row in dataFrame.iterrows():
        dataFrame.at[index, 'id'] = id
        id += 1
    
    return dataFrame

In [27]:
# Transform truth values from six classes to two. 'True' and 'False'

trueLabels  = ['mostly-true', 'true']
falseLabels = ['pants-fire', 'false', 'barely-true', 'half-true']

trueData, falseData = transformTruthValues(trainData, trueLabels, falseLabels)

In [28]:
# Set pandas options show that dataframes appear not truncated.
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [30]:
start = 0
step  = 50

for i in range(1,6):
    # Fix ids
    data = fixIds(falseData[start:start+step])
    # Transform data to json string
    evalData  = data.to_json(orient="records")
    # Write as json to file
    evaluation = open("evaluation" + str(i) + ".json", "w")
    evaluation.write(json.dumps(json.loads(evalData), indent=4, sort_keys=True))
    evaluation.close()
    
    start = start + step